<a href="https://colab.research.google.com/github/CSSamarasinghe/SE4050_Assignment/blob/IT21263194/After_dataset_change_improvement_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'amazon-reviews-for-sentianalysis-finegrained-csv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2078107%2F3499094%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241003%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241003T164549Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D25a30a2ad799910d48f45f67793fb1d09f31560ae3773fa14870108018fa213261ae62aeb020867952a30d8094a21359c2546a6475193ac986bc2645a191d848c7223dbd5c68600aeb8f17d5a979be9750827328816b6743cf0d74ca79e5a28878f8606f4d3cd2176678d18b8a70a20bbf9be8037aa238d4b56bd1a0ba1a8d1fb4003bbd12f51fa6d438aaa8a7e8fa90a9291e62969f7df78c34dc6a71c76fb2f1c11e2964127136ec4d5554e9f1bdd7c530f8ddf987a08dd6cb91a849e4e98a2089149d22cb00ca687688c756071af4270e392fd79699c1f4a35ca16597523ca8bfe37fea11d5911465ea51d6eb17c1e3663da806763c8ef4e96e1f0a9ae9ab'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

[==================================================] 654512809 bytes downloaded
Downloaded and uncompressed: amazon-reviews-for-sentianalysis-finegrained-csv
Data source import complete.


In [2]:

import pandas as pd

# Load the dataset
train = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')
test = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/test.csv')

# Check the column names
print("Training data set")
print(train.columns)
print(train.shape)

print("Testing data set")
print(test.columns)
print(test.shape)

Training data set
Index(['class_index', 'review_title', 'review_text'], dtype='object')
(3000000, 3)
Testing data set
Index(['class_index', 'review_title', 'review_text'], dtype='object')
(650000, 3)


In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from tensorflow.keras.preprocessing.text import Tokenizer
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import OneHotEncoder

# Load the dataset
train_data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/train.csv')
test_data = pd.read_csv('../input/amazon-reviews-for-sentianalysis-finegrained-csv/amazon_review_fine-grained_5_classes_csv/test.csv')

# Reduce the dataset to a manageable size (e.g., taking the first 1000 samples)
train_data = train_data.sample(n=25000, random_state=42)  # Randomly sample 1000 rows from the training data
test_data = test_data.sample(n=5000, random_state=42)      # Randomly sample 200 rows from the test data

# Initialize the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['review_text'])

# Convert text to sequences
X_train = tokenizer.texts_to_sequences(train_data['review_text'])
X_test = tokenizer.texts_to_sequences(test_data['review_text'])

# Directly take the numeric values from class_index
Y_train = train_data['class_index'].values
Y_test = test_data['class_index'].values

# If needed, one-hot encode the labels for multi-class classification
Y_train_reshaped = Y_train.reshape(-1, 1)
Y_test_reshaped = Y_test.reshape(-1, 1)

# Use sparse_output instead of sparse
encoder = OneHotEncoder(sparse_output=False)
Y_train_onehot = encoder.fit_transform(Y_train_reshaped)
Y_test_onehot = encoder.transform(Y_test_reshaped)

# Print the shapes to confirm
print(f'X_train shape: {len(X_train)}, Y_train shape: {Y_train_onehot.shape}')
print(f'X_test shape: {len(X_test)}, Y_test shape: {Y_test_onehot.shape}')

# Pad the sequences and convert to tensors
X_train_padded = pad_sequence([torch.tensor(seq) for seq in X_train], batch_first=True, padding_value=0)
X_test_padded = pad_sequence([torch.tensor(seq) for seq in X_test], batch_first=True, padding_value=0)

# Adjust the input size
input_size = len(tokenizer.word_index) + 1  # Vocabulary size (plus 1 for padding)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_padded, torch.tensor(Y_train_onehot, dtype=torch.float32))
test_dataset = TensorDataset(X_test_padded, torch.tensor(Y_test_onehot, dtype=torch.float32))

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)



X_train shape: 25000, Y_train shape: (25000, 5)
X_test shape: 5000, Y_test shape: (5000, 5)


In [4]:
import torch
import torch.nn as nn

# LSTM Model Definition
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x = torch.tanh(x)
        x, _ = self.lstm2(x)
        x = self.fc(x[:, -1, :])  # Output logits for the last time step
        return x


In [5]:
# Set hyperparameters
num_epochs = 5
embedding_dim = 128
hidden_size = 64
output_size = len(set(Y_train))  # Number of classes
learning_rate = 0.001

# Initialize the model **before** defining the optimizer
model = LSTMModel(vocab_size=input_size, embedding_dim=embedding_dim, hidden_size=hidden_size, output_size=output_size)

In [6]:
import torch.optim as optim
from torch.utils.data import DataLoader


# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()  # Clear the gradients
        inputs = inputs.long()  # Ensure inputs are long type for embedding
        outputs = model(inputs)  # Forward pass

        # Compute loss (labels are class indices)
        loss = criterion(outputs, labels)  # Labels should be LongTensor of class indices

        # Backward pass and optimization
        loss.backward()  # Backward pass
        optimizer.step()  # Optimization step

    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/5], Loss: 1.6168
Epoch [2/5], Loss: 1.6113
Epoch [3/5], Loss: 1.6004
Epoch [4/5], Loss: 1.6138
Epoch [5/5], Loss: 1.6067


Experimenting with different learning rates

In [9]:
import numpy as np

# Define a range of learning rates to test
learning_rates = [0.0001, 0.001, 0.01, 0.1]

# Store results for analysis
results = {}

for lr in learning_rates:
    # Initialize the model
    model = LSTMModel(vocab_size=input_size, embedding_dim=embedding_dim, hidden_size=hidden_size, output_size=output_size)

    # Set up loss function and optimizer with the current learning rate
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Train the model for a few epochs (e.g., 3 epochs for quick testing)
    for epoch in range(3):
        model.train()
        total_loss = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Clear gradients
            inputs = inputs.long()  # Ensure inputs are long type for embedding

            outputs = model(inputs)  # Forward pass

            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        results[lr] = results.get(lr, []) + [avg_loss]
        print(f'Learning Rate: {lr}, Epoch: {epoch + 1}, Average Loss: {avg_loss:.4f}')

# Analyze results
for lr, losses in results.items():
    print(f'Learning Rate: {lr}, Losses: {losses}')

Learning Rate: 0.0001, Epoch: 1, Average Loss: 1.6099
Learning Rate: 0.0001, Epoch: 2, Average Loss: 1.6098
Learning Rate: 0.0001, Epoch: 3, Average Loss: 1.6097
Learning Rate: 0.001, Epoch: 1, Average Loss: 1.6105
Learning Rate: 0.001, Epoch: 2, Average Loss: 1.6099
Learning Rate: 0.001, Epoch: 3, Average Loss: 1.6098
Learning Rate: 0.01, Epoch: 1, Average Loss: 1.6108
Learning Rate: 0.01, Epoch: 2, Average Loss: 1.6105
Learning Rate: 0.01, Epoch: 3, Average Loss: 1.6105
Learning Rate: 0.1, Epoch: 1, Average Loss: 1.6249
Learning Rate: 0.1, Epoch: 2, Average Loss: 1.6174
Learning Rate: 0.1, Epoch: 3, Average Loss: 1.6170
Learning Rate: 0.0001, Losses: [1.6099145712754916, 1.6098479453255148, 1.6096684926611078]
Learning Rate: 0.001, Losses: [1.6104630245577038, 1.6098775278271922, 1.6097668683742319]
Learning Rate: 0.01, Losses: [1.6108124612847252, 1.6104681990335665, 1.6104737291555575]
Learning Rate: 0.1, Losses: [1.62490014042086, 1.6173984196484852, 1.6169835295518646]



Experiment with different sample size

> Add blockquote



In [10]:
# Define a range of batch sizes to test
batch_sizes = [16, 32, 128]

# Store results for analysis
batch_results = {}

for batch_size in batch_sizes:
    # Create DataLoader with the current batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    # Initialize the model again (use optimal learning rate found earlier)
    model = LSTMModel(vocab_size=input_size, embedding_dim=embedding_dim, hidden_size=hidden_size, output_size=output_size)

    # Set up loss function and optimizer with an optimal learning rate (e.g., from previous step)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)  # Replace with optimal lr if found

    # Train the model for a few epochs (e.g., 3 epochs for quick testing)
    for epoch in range(3):
        model.train()
        total_loss = 0

        for inputs, labels in train_loader:
            optimizer.zero_grad()  # Clear gradients
            inputs = inputs.long()  # Ensure inputs are long type for embedding

            outputs = model(inputs)  # Forward pass

            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backward pass
            optimizer.step()  # Update weights

            total_loss += loss.item()

        avg_loss = total_loss / len(train_loader)
        batch_results[batch_size] = batch_results.get(batch_size, []) + [avg_loss]
        print(f'Batch Size: {batch_size}, Epoch: {epoch + 1}, Average Loss: {avg_loss:.4f}')

# Analyze results
for batch_size, losses in batch_results.items():
    print(f'Batch Size: {batch_size}, Losses: {losses}')

Batch Size: 16, Epoch: 1, Average Loss: 1.6102
Batch Size: 16, Epoch: 2, Average Loss: 1.6100
Batch Size: 16, Epoch: 3, Average Loss: 1.6097
Batch Size: 32, Epoch: 1, Average Loss: 1.6103
Batch Size: 32, Epoch: 2, Average Loss: 1.6098
Batch Size: 32, Epoch: 3, Average Loss: 1.6096
Batch Size: 128, Epoch: 1, Average Loss: 1.6103
Batch Size: 128, Epoch: 2, Average Loss: 1.6097
Batch Size: 128, Epoch: 3, Average Loss: 1.6098
Batch Size: 16, Losses: [1.6102401017380006, 1.6100081010118021, 1.609708639420688]
Batch Size: 32, Losses: [1.6103278011312265, 1.6097525771316665, 1.6095811293253204]
Batch Size: 128, Losses: [1.6102710512219642, 1.6097411884337056, 1.6097593307495117]


In [8]:
import matplotlib.pyplot as plt

# Assuming you have stored the training loss and accuracy in lists
train_losses = []
train_accuracies = []

# During your training loop, you can append loss and accuracy values
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    correct_predictions = 0
    total_samples = 0

    for inputs, labels in train_loader:
        optimizer.zero_grad()
        inputs = inputs.long()
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total_samples += labels.size(0)
        correct_predictions += (predicted == labels.argmax(dim=1)).sum().item()

    avg_loss = epoch_loss / len(train_loader)
    accuracy = correct_predictions / total_samples

    train_losses.append(avg_loss)
    train_accuracies.append(accuracy)

# After training, evaluate the model on the test set
test_loss, test_accuracy = evaluate_model(model, test_loader)

# Plotting Loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss', color='blue')
plt.title('Training Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plotting Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Training Accuracy', color='green')
plt.title('Training Accuracy Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Print test results
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

KeyboardInterrupt: 

In [7]:
# Evaluation function
def evaluate_model(model, data_loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for inputs, labels in data_loader:
            inputs = inputs.long()  # Ensure inputs are long type for embedding
            outputs = model(inputs)  # Forward pass

            # Compute loss
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Get predicted class indices
            _, predicted = torch.max(outputs, 1)
            total_samples += labels.size(0)
            correct_predictions += (predicted == labels.argmax(dim=1)).sum().item()

    avg_loss = total_loss / len(data_loader)
    accuracy = correct_predictions / total_samples

    return avg_loss, accuracy

# Evaluate on the test set
test_loss, test_accuracy = evaluate_model(model, test_loader)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

Test Loss: 1.6098, Test Accuracy: 0.2034
